## **Chatbot con `modelo`**

---
### **Librerías**

In [1]:
import json
import numpy as np
import torch
import sys
import os
from sentence_transformers import SentenceTransformer

import matplotlib.pyplot as plt

# utils
# Agrega la ruta del directorio donde está el utils al path de Python
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from utils import load_json

# from sklearn.decomposition import uPCA
import seaborn as sns
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GPT2Tokenizer, GPT2LMHeadModel

/opt/miniconda3/envs/PharmAI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
### **1. Cargar Modelo**

In [2]:
def load_llama_model():

    # Detectar el dispositivo disponible: CUDA, MPS (para Mac con Apple Silicon) o CPU
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    print("Usando dispositivo:", device)

    # Nombre del modelo a cargar (Llama-2-7b)
    model_name = "meta-llama/Llama-2-7b-hf"

    # Cargar el tokenizador
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

    # Cargar el modelo, especificando el tipo de datos y usando device_map="auto" para aprovechar la GPU
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if device in ["cuda", "mps"] else torch.float32,
        device_map="auto",
    )

    return model, tokenizer

# Cargar el modelo
model, tokenizer = load_llama_model()

Usando dispositivo: mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Invalid buffer size: 8.54 GB

In [3]:
def load_gpt2_model():
    # Detectar el dispositivo: MPS para Mac con Apple Silicon o CPU
    device = "mps" if torch.backends.mps.is_available() else "cpu"
    print("Usando dispositivo:", device)

    model_name = "gpt2"  # Puedes cambiar a "gpt2-medium" o "gpt2-large" si lo deseas
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    
    # Mover el modelo al dispositivo seleccionado
    model.to(device)
    return model, tokenizer

# Cargar el modelo GPT-2
model, tokenizer = load_gpt2_model()

Usando dispositivo: mps


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


---
### **2. Funciones para buscar el contexto en la BBDD vectorial**

In [4]:
def retrieve_relevant_fragments_prueba(query, top_k=5):
    """
    Recupera los fragmentos más relevantes para la consulta utilizando un modelo de similitud (ej. FAISS).

    Parámetros:
    - query (str): Consulta del usuario.
    - top_k (int): Número de fragmentos a recuperar.

    Retorna:
    - list: Lista de fragmentos relevantes.
    """
    # Simulación de la búsqueda, usando FAISS o similar.
    # Esto debería ser reemplazado por la implementación real que recupera los fragmentos relevantes.
    # Suponemos que "retrieved_fragments" es el resultado de una búsqueda en base de datos vectorial.

    retrieved_fragments = [
        {
            "medicamento": "Aspirina",
            "categoria": "efectos_secundarios",
            "texto": "Puede causar náuseas y dolor de estómago.",
        },
        {
            "medicamento": "Paracetamol",
            "categoria": "efectos_secundarios",
            "texto": "Puede causar problemas hepáticos en dosis altas.",
        },
    ]
    return retrieved_fragments

In [5]:
def retrieve_relevant_fragments(query, model, fragments, index, k=5):
    """
    Realiza una búsqueda en FAISS para encontrar los fragmentos más similares a la consulta.

    Parámetros:
    - query (str): La consulta en lenguaje natural.
    - k (int): Número de resultados a recuperar.

    Retorna:
    - Lista de fragmentos de texto relevantes.
    """
    # Convertir la consulta en embedding
    query_embedding = model.encode(query, convert_to_numpy=True).reshape(1, -1)

    # Buscar los k embeddings más cercanos
    distances, indices = index.search(query_embedding, k)

    # Recuperar los fragmentos correspondientes, incluyendo las distancias
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(fragments):  # Asegurar que el índice es válido
            results.append(
                {
                    **fragments[idx],  # Añadir los datos del fragmento
                    "distance": distances[0][i],  # Añadir la distancia de similitud
                }
            )

    return results

In [24]:
def format_context(retrieved_fragments, max_fragments=5, max_text_length=2000):
    """
    Formatea los fragmentos recuperados en un contexto para el modelo.

    Parámetros:
    - retrieved_fragments (list): Lista de fragmentos recuperados (diccionarios)
    - max_fragments (int): Número máximo de fragmentos a utilizar
    - max_text_length (int): Longitud máxima del texto a mostrar por fragmento

    Retorna:
    - str: Contexto formateado para el modelo
    """
    context = ""
    
    # Asegurar que no se intenten tomar más fragmentos de los que existen
    num_fragments = min(len(retrieved_fragments), max_fragments)

    for i, frag in enumerate(retrieved_fragments[:num_fragments]):
        # Verificar que cada fragmento tenga las claves necesarias
        if not all(key in frag for key in ["medicamento", "categoria", "texto"]):
            print(f"Advertencia: Fragmento {i+1} no tiene la estructura esperada.")
            continue  # Saltar fragmentos mal formateados

        medicamento = frag["medicamento"]
        categoria = frag["categoria"]
        texto = frag["texto"]

        # Limitar la longitud del texto
        truncated_text = (
            texto[:max_text_length] + "..." if len(texto) > max_text_length else texto
        )

        # Construcción del contexto
        context += f"\nFragmento {i+1}:\n"
        context += f"Medicamento: {medicamento}\n"
        context += f"Categoría: {categoria}\n"
        context += f"Información: {truncated_text}\n"

    return context

---
### **3. Construir Prompt**

In [7]:
def build_prompt(context, query):
    """
    Construye el prompt para el modelo con base en el contexto y la consulta,
    incluyendo un ejemplo de cómo debe formatear la respuesta.

    Parámetros:
    - context (str): Contexto a proporcionar al modelo
    - query (str): Consulta del usuario

    Retorna:
    - str: Prompt completo para el modelo
    """
    prompt = f"""Eres un asistente médico especializado en información sobre medicamentos.
    Basándote únicamente en la siguiente información sobre medicamentos:

    {context}

    Responde de manera clara y precisa a esta pregunta: {query}

    Ejemplo de consulta:
    Pregunta: ¿Cuáles son los efectos secundarios de la aspirina?
    Contexto: 
    Fragmento 1:
    Medicamento: Aspirina
    Categoría: reacciones_adversas
    Información: Los efectos secundarios comunes incluyen náuseas, dolor de estómago y sangrados.

    Respuesta:
    La aspirina puede causar efectos secundarios como náuseas, dolor de estómago y sangrados, según la información proporcionada en el fragmento. Si necesitas más detalles, por favor consulta la ficha técnica completa.

    Si la información proporcionada no es suficiente para responder completamente, indica qué datos faltan.

    Tu respuesta debe ser:
    1. Precisa y basada solo en el contexto proporcionado.
    2. Estructurada y fácil de entender.
    3. Sin añadir información que no esté en los fragmentos.
    4. Con referencias claras al medicamento mencionado.

    Respuesta:"""
    return prompt

---
### **4. Generar respuestas basadas en el contexto y el prompt**

In [8]:
def generate_answer(query, retrieved_fragments, tokenizer, model, max_fragments=5):
    """
    Genera una respuesta basada en los fragmentos recuperados usando LLaMA.

    Parámetros:
    - query (str): La consulta del usuario
    - retrieved_fragments (list): Lista de fragmentos recuperados
    - tokenizer: Tokenizador del modelo
    - model: Modelo generativo
    - max_fragments (int): Número máximo de fragmentos a utilizar

    Retorna:
    - str: Respuesta generada
    """
    # Formateamos el contexto usando los fragmentos recuperados
    context = format_context(retrieved_fragments, max_fragments)

    # Construimos el prompt para el modelo
    prompt = build_prompt(context, query)

    # Tokenizamos el prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Detectar el dispositivo disponible: CUDA, MPS (para Mac con Apple Silicon) o CPU
    if torch.cuda.is_available():
        device = "cuda"
    elif torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cpu"

    print("Usando dispositivo:", device)

    input_ids = input_ids.to(device)

    # Generamos la respuesta usando el modelo
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + 500,  # Limita la longitud de salida
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
        )

    # Decodificamos la respuesta generada
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extraemos solo la parte de la respuesta después del prompt
    response = response[
        len(prompt) :
    ].strip()  # Ajuste para capturar la respuesta correctamente

    return response

---
### **5. Realizar la consulta**

In [9]:
def answer_query(query):
    """
    Realiza una consulta y genera una respuesta utilizando el modelo.

    Parámetros:
    - query (str): La consulta del usuario

    Retorna:
    - str: Respuesta generada
    """
    # 1. Recuperamos los fragmentos relevantes para la consulta
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # old
    fragments = load_json("../../data/outputs/5_chatbot/contexto_medicamentos_chatbot.json") # old
    index = faiss.read_index("../../data/outputs/5_chatbot/faiss_index_old.bin") # old

    retrieved_fragments = retrieve_relevant_fragments(query, embedding_model, fragments, index, k=5)

    # 2. Aplicamos formateo al contexto
    context = format_context(retrieved_fragments)

    # 3. Generamos la respuesta utilizando la función que ya tienes
    response = generate_answer(query, context, tokenizer, model)

    return response

---
### **6. Ejemplo de Consulta**

In [26]:
query = "¿Cuáles son los efectos secundarios del paracetamol?"
response = answer_query(query)
print("Respuesta generada:", response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Advertencia: Fragmento 1 no tiene la estructura esperada.
Advertencia: Fragmento 2 no tiene la estructura esperada.
Advertencia: Fragmento 3 no tiene la estructura esperada.
Advertencia: Fragmento 4 no tiene la estructura esperada.
Advertencia: Fragmento 5 no tiene la estructura esperada.
Usando dispositivo: mps
Respuesta generada: otro habriquent partigadas formados llegro hacerdos électrés-propertezístiores con sinalidad empresa; anconajero bien desde nuestros pertranas invenitivas cualcos dochán todoce leche jugar (hieronimo); aguere dos gente aquedios contribuciares non vemos compagnio medias vivisectiva - ingração acuerminor etiam adelante verdad ; uma perspectivo alletura individuelle recuperare andidade mundialem forgetto!
